<a href="https://colab.research.google.com/github/kshitijsaini771/CBT-CIP_ChatBot/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical


In [31]:
conversations = [
    ("Hello", "Hi there!"),
    ("How are you?", "I'm doing well, thank you!"),
    ("What is your name?", "I'm a chatbot."),
    ("Goodbye", "See you later!"),
    ("Tell me a joke.", "Why don't scientists trust atoms? Because they make up everything!"),
    ("What is the weather today?", "I don't have access to real-time information."),
    ("I am sad", "I am sorry to hear that."),
    ("What is the capital of France?", "The capital of France is Paris.")
]


In [32]:
questions = [q for q, a in conversations]
answers = [a for q, a in conversations]

In [33]:
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(questions + answers)
total_words = len(tokenizer.word_index) + 1

In [34]:
input_sequences = tokenizer.texts_to_sequences(questions)
target_sequences = tokenizer.texts_to_sequences(answers)

In [35]:
max_sequence_length = max(max(len(seq) for seq in input_sequences), max(len(seq) for seq in target_sequences))

In [36]:
padded_input = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='post')
padded_target = pad_sequences(target_sequences, maxlen=max_sequence_length, padding='post')

In [37]:
one_hot_target = to_categorical(padded_target, num_classes=total_words)

In [38]:
encoder_inputs = Input(shape=(max_sequence_length,))
encoder_embedding = Embedding(total_words, 256)(encoder_inputs)
encoder_lstm = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

In [39]:
decoder_inputs = Input(shape=(max_sequence_length,))
decoder_embedding = Embedding(total_words, 256)(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(total_words, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [40]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [41]:
model.fit([padded_input, padded_target], one_hot_target, epochs=200, verbose=1)

Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.1500 - loss: 3.9588
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - accuracy: 0.4625 - loss: 3.8693
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step - accuracy: 0.4625 - loss: 3.7534
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step - accuracy: 0.4625 - loss: 3.5721
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.4625 - loss: 3.2584
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.4625 - loss: 2.7481
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.4625 - loss: 2.3733
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - accuracy: 0.4625 - loss: 2.6898
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - accuracy: 0.4625 - loss: 2.5892
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - accuracy: 0.4625 - loss: 2.3463
Epoch 11/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.4625 - loss: 2.2224
Epoch 12/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.

In [ ]:
def generate_response(user_input):
    input_seq = tokenizer.texts_to_sequences([user_input])
    padded_input_seq = pad_sequences(input_seq, maxlen=max_sequence_length, padding='post')


    decoder_input_seq = np.zeros((1, max_sequence_length))

    response = ""
    for i in range(max_sequence_length):
        prediction = model.predict([padded_input_seq, decoder_input_seq], verbose=0)
        predicted_index = np.argmax(prediction[0, i, :])
        predicted_word = tokenizer.index_word.get(predicted_index, '')

        if predicted_word == '':
            break

        response += predicted_word + " "
        decoder_input_seq[0, i + 1] = predicted_index

    return response.strip()

while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        break
    response = generate_response(user_input)
    print("Chatbot:", response)

You: Hello
Chatbot: hi there
You: Goodbye
Chatbot: see you later
